<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Предобработка</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Исходные-данные--без-масштабирования" data-toc-modified-id="Исходные-данные--без-масштабирования-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Исходные данные  без масштабирования</a></span></li><li><span><a href="#Исходные-данные---масштабированные" data-toc-modified-id="Исходные-данные---масштабированные-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Исходные данные   масштабированные</a></span></li><li><span><a href="#После-преобразования-без-масштабирования" data-toc-modified-id="После-преобразования-без-масштабирования-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>После преобразования без масштабирования</a></span></li><li><span><a href="#После-преобразования-масштабированные" data-toc-modified-id="После-преобразования-масштабированные-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>После преобразования масштабированные</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
# импортируем библиотеки
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
# Откроем и прочитаем файл
pth1 = 'datasets/insurance.csv'
pth2 = '/datasets/insurance.csv'


if os.path.exists(pth1):
    data = pd.read_csv(pth1)
elif os.path.exists(pth1):
    data = pd.read_csv(pth2)
else:
    print('wrong somethings')

In [3]:
#Константа
RANDOM_STATE = 12345

In [4]:
# Посмотрим данные
data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [5]:
#Посмотрим общую информацию
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


**Вывод:** Пропусков нет. Необходимо переименовать названия столбцов, и преобразовать данные типа float к типу int64 

### Предобработка

In [7]:
#ПЕременовываем названия столбцов
data = data.rename(columns = {
    'Пол': 'gender',
    'Возраст': 'age',
    'Зарплата': 'salary',
    'Члены семьи': 'family_members',
    'Страховые выплаты': 'insurance_payments'
    }
)

In [8]:
#Приводим к типу int64
data[['age', 'salary']] = data[['age', 'salary']].astype('int64')

In [9]:
# Проверка на дубликаты
data.duplicated(subset=data.columns,).sum()

153

In [10]:
#Посмотрим на дубликаты
data[data.duplicated(subset=data.columns, keep=False)==True].sort_values('salary')

,gender,age,salary,family_members,insurance_payments
2955,1,32,21600,0,0
2988,1,32,21600,0,0
361,0,50,24700,1,2
2869,0,50,24700,1,2
333,0,32,25600,1,0
...,...,...,...,...,...
2465,0,40,54000,2,0
1042,0,33,59300,2,0
2231,0,33,59300,2,0
1418,1,28,59900,0,0


In [11]:
#Удаление дубликатов
data = data.drop_duplicates(subset=data.columns)
data.duplicated(subset=data.columns).sum()

0

**Вывод:** Данные подготовленны для дальнейшего изучения и использования

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$


Предсказания после умножения на обратимую матрицу:

$$
a_p = XPw'
$$

Т.к. на обратимую матрицу умножается Х, то меняется и w:

$$
w' = ((XP)^T(XP))^{-1} (XP)^T y
$$

Нужно доказать что:

$$
a_p = a
$$

Для доказательства умножим a на единичную матрицу Е, т.к. мы знаем что, если  любую
матрицу  умножить на единичную (или наоборот), получится эта же
матрица :

$$
 a = a E
$$
 $$
 E = PP^{-1}
$$
$$
 а = XPP^{-1}w = (XP)P^{-1}w
$$

Докажем что:

$$
 w' = P^{-1}w
$$

Доказательство:

$$
w' = ((XP)^T(XP))^{-1} (XP)^T y = (P^T (X^T X) P)^{-1} P^T X^T y 
$$
$$
 w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^Ty
$$
$$
w'= P^{-1}(X^T X)^{-1}X^Ty
$$
$$
w' = P^{-1}w
$$

Следовательно:
$$
 a = XPw'
$$

Что и требовалось доказать.

**Вопрос:**  Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 

**Ответ:** Не иизменится. Качество предсказаний будет таким же.

**Обоснование:** Смотреть выше 👆

## Алгоритм преобразования

**Алгоритм**

Для защиты данных мы будем умножать матрицу признаков на обратимую матрицу, которая будет генерироваться случайно. При этом матрица должна быть квадратичной т.е. размер матрицы должен быть `[n x n]`. Для этого напишем функцию которая генерирует обратимую матрицу.
Алгоритм работы функции для нахождения обратимой матрицы преобразования:
1. Функция на вход получает данные размерностью `[m x n]`.
2. Находим размерность матрицы преобразования, она равна ширине полученных данных `n`
3. Генерируем рандомную  матрицу преобразования размером `[n x n]`
4. Проверяем существует ли для нее обратная матрица:
     - a. Обратная матрица существует, функция возвращает матрицу преобразования
     - b. Обратная матрица несуществует, функция вызывает саму себя и падает в рекурсию, пока не выполниться подпункт а.

Получив матрицу преобразований, умножаем ее на полученные данные. Перед эти предварительно преобразуем данные в матрицу с помощью метода `.values` 


In [12]:
def generate_transform_matrix(features):
    n = features.shape[1]
    transform_matrix = np.random.randint(1,10, (n,n))
    try:
        inverse = np.linalg.inv(transform_matrix)
        return transform_matrix
    except numpy.linalg.LinAlgError:
       generate_transform_matrix(features)
    

In [13]:
def encryptor(features, transform_matrix):
    features = features.values @ transform_matrix
    return features

**Обоснование:**
Посмотрите доказательство идентичности предсказаний модели в пункте 2.

## Проверка алгоритма

**Для линейной регрессии данные на вход обычно дают стандартизированные, поэтом у мы будем проверять модель в 4 этапах:**
- Исходные данные без масштабирования 
- Исходные данные масштабированные
- После преобразования без масштабирования
- После преобразования масштабированные


In [14]:
#Определим признаки и целевой признак
features = data.drop('insurance_payments', axis=1)
target = data['insurance_payments']

In [15]:
#Понадобиться для будующей таблицы метрик. Сохряняем сюда значение R2 после каждого этапа.
score = []

### Исходные данные  без масштабирования

In [16]:
# Разбивка на тренировочную и валидационную.
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE)

#Обучаем модель и смотрим метрику R2
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)

score.append(r2_score(target_valid, predictions))
print('R2 score =', r2_score(target_valid, predictions))

R2 score = 0.4230772761583639


###  Исходные данные   масштабированные

In [17]:
#Масштабируем
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)

#Обучаем модель и смотрим метрику R2
model.fit(features_train, target_train)
predictions = model.predict(features_valid)

score.append(r2_score(target_valid, predictions))
print('R2 score =', r2_score(target_valid, predictions))

R2 score = 0.4230772761583671


### После преобразования без масштабирования

In [18]:
#Преобразуем features
transform_matrix = generate_transform_matrix(features)
transform_features = encryptor(features, transform_matrix)

#Напечатаем матрицу преобразования
transform_matrix

array([[4, 4, 5, 7],
       [4, 9, 5, 5],
       [7, 8, 6, 1],
       [7, 8, 3, 5]])

In [19]:
#Напечатаем измененные признаки
transform_features

array([[347375, 397181, 297813,  49817],
       [266191, 304422, 228233,  38235],
       [147116, 168261, 126145,  21145],
       ...,
       [237394, 271396, 203506,  34010],
       [229013, 261826, 196324,  32832],
       [284323, 325064, 243748,  40752]], dtype=int64)

In [20]:
# Разбивка на тренировочную и валидационную.
transform_features_train, transform_features_valid, transform_target_train, transform_target_valid = train_test_split(
    transform_features, target, test_size=0.25, random_state=RANDOM_STATE)

#Обучаем модель и смотрим метрику R2
model.fit(transform_features_train, transform_target_train)
predictions = model.predict(transform_features_valid)

score.append(r2_score(transform_target_valid, predictions))
print('R2 score =', r2_score(transform_target_valid, predictions))

R2 score = 0.4230772761584144


### После преобразования масштабированные

In [21]:
#Масштабируем
scaler.fit(transform_features_train)
transform_features_train = scaler.transform(transform_features_train)
transform_features_valid = scaler.transform(transform_features_valid)

#Обучаем модель и смотрим метрику R2
model.fit(transform_features_train, transform_target_train)
predictions = model.predict(transform_features_valid)
score.append(r2_score(transform_target_valid, predictions))
print('R2 score =', r2_score(transform_target_valid, predictions))

R2 score = 0.423077276158353


### Выводы

In [22]:
#Напечатаем таблицу изменений качества метрики
index = ['Исходные данные без масштабирования','Исходные данные масштабированные', 
       'После преобразования без масштабирования', 'После преобразования масштабированные']
conclusion = pd.DataFrame(score, columns=['R2_score'], index=index)
conclusion

,R2_score
Исходные данные без масштабирования,0.423077
Исходные данные масштабированные,0.423077
После преобразования без масштабирования,0.423077
После преобразования масштабированные,0.423077


**Выводы**: Как мы можем можем наблюдать после шифровки данных качество неизменилось. Теоретические выкладки получили практическое подтверждение

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные загружены
- [ ]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [ ]  Указан правильный вариант ответа
    - [ ]  Вариант обоснован
- [ ]  Выполнен шаг 3: предложен алгоритм преобразования
    - [ ]  Алгоритм описан
    - [ ]  Алгоритм обоснован
- [ ]  Выполнен шаг 4: алгоритм проверен
    - [ ]  Алгоритм реализован
    - [ ]  Проведено сравнение качества моделей до и после преобразования